# Cell Recognition Explorer
Este notebook é dedicado à experimentação passo a passo das etapas de leitura das imagens, aplicação de algoritmos de pré-processamento até a estimação do número de células via Transformada de Hough

In [ ]:
import pandas as pd
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Confirmar versões carregadas das bibliotecas
print('OpenCV version:',cv2.__version__)

# Análise exploratória das imagens do dataset
## Visualização da imagem com seus canais de cor

In [ ]:
my_dpi = 96
#arquivos_experimento = ['BloodImage_00023.jpg']
caminho_dataset = '../dataset/BCCD/JPEGImages/'
#escolha de uma imagem arbitrária para visualização e experimentação
img_selecionada = 'BloodImage_00023.jpg'
arquivo = '../dataset/BCCD/JPEGImages/' + img_selecionada

img_original = cv2.imread(arquivo)
#converte para o esquema de cores RGB, pois o opencv carrega em formato BGR
img_rgb = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
#mostra imagem original
plt.imshow(img_rgb)


In [ ]:
#exibe o shape da imagem (resolução e canais de cor)
print(img_rgb.shape)

In [ ]:
#separa matrizes pelos canais de cor
blue, green, red = cv2.split(img_original)

### Histograma de cores
Neste ponto estamos interessados em visualizar o histograma de cores para uma imagem do dataset

In [ ]:
#Exibe o histograma dos canais de cores para a imagem
fig, axs = plt.subplots(1,3, constrained_layout=True, sharey=True)
axs[0].hist(red.ravel(),256,[0,256],color='red')
axs[0].set_title('Vermelho (R)')
axs[1].hist(green.ravel(), 256,[0,256], color='green')
axs[1].set_title('Verde (G)')
axs[2].hist(blue.ravel(), 256,[0,256], color='blue')
axs[2].set_title('Azul (B)')
fig.suptitle('Histograma dos canais de cores')
plt.show()

In [ ]:
#Mostrando cada canal em escala de cinza:
fig2, ax2 = plt.subplots(1,3,constrained_layout=True, sharey=True,figsize=(12,16))
ax2[0].imshow(red, cmap='gray')
ax2[0].set_title('Vermelho')
ax2[1].imshow(green,cmap='gray')
ax2[1].set_title('Verde')
ax2[2].imshow(blue,cmap='gray')
ax2[2].set_title('Azul')


## Conversão da imagem para escala em tons de cinza

In [ ]:
#converte a imagem para tons de cinza
img_gray = cv2.cvtColor(img_original,cv2.COLOR_BGR2GRAY)
#exibe imagem em tons de cinza
plt.imshow(img_gray,cmap='gray')
plt.show()

In [ ]:
img_ruido = cv2.imread('../dataset/images/tons_cinza_ruido.jpg',0)
print(img_ruido)
plt.imshow(img_ruido,cmap='gray')

In [ ]:
#obtem uma secao de uma imagem com ruido. Confirma as dimensões da imagem, em formato grayscale
img_ruido.shape

In [ ]:
img_secao_ruido = img_ruido[0:10,0:10]
img_secao_norm  = img_gray[0:10,0:10]
fig3, ax3 = plt.subplots(1,2,constrained_layout=True, sharey=True,figsize=(10,10))

ax3[0].imshow(img_secao_norm, cmap='gray')
ax3[0].set_title('Seção 9x9 da imagem original')

ax3[1].imshow(img_secao_ruido, cmap='gray')
ax3[1].set_title('Seção 9x9 da imagem com ruído')

## Histograma para tons de cinza
Após a análise do histograma de cores, vamos observar o histograma da imagem em tons de cinza

In [ ]:
#Vamos analisar o histograma da imagem para avaliar a ocorrência da intensidade
intensidades = img_gray.ravel()
plt.hist(intensidades,256,[0,256])
plt.show()

In [ ]:
#Analisando por bloxplot
plt.boxplot(intensidades);

In [ ]:
#obtendo as medidas estatísticas básicas das intensidades
pd_intensidades = pd.DataFrame(intensidades)
pd_intensidades.describe()

## Redução de ruído (Gauss, Mediana e Média)
Neste ponto estamos interessados em aplicar os três métodos e comparar os resultados

In [ ]:
def noise_reduction(img_orig, noise_type):
    if noise_type == 'gauss':
        #parametros ajustados
        kern = (3,3) #tamanho do Kernel
        #aplicação do algoritmo sobre a imagem
        img_non_noisy = cv2.GaussianBlur(img_orig,kern,cv2.BORDER_DEFAULT)
    elif noise_type == 'median':
        #parametros ajustados
        kern_size = 5
        img_non_noisy = cv2.medianBlur(img_orig, kern_size)
    elif noise_type == 'mean':
        #parametros ajustados
        kern = (3,3)
        img_non_noisy = cv2.blur(img_orig,kern)
    return img_non_noisy

In [ ]:
# Define parâmetros para a Transformada de Hough e outros
minRadius_esc = 24 #menor raio de célula
maxRadius_esc = 58 #maior raio de célula (anterior, usado no cálculo =58)
param1_esc = 20    #threshold escolhido para Canny
param2_esc = 22    #menor distância entre dois centros

#filtro escolhido
filtro_esc = 'mean'

#output das imagens
caminho_output  = '../output/'

## Testa as diferentes combinações de filtros

In [ ]:
# carrega dataframe com a lista de imagens
df_rbc = pd.read_csv('quantidade_celulas_rbc.csv')

In [ ]:
# cria lista de filtros
filter_set = ['gauss','median','mean']
resultado = []
for f in filter_set:
    for row in df_rbc.iterrows():
        
        #percorre cada arquivo do dataframe
        arquivo = caminho_dataset + row[1][0]
        
        #para este arquivo anota o ground truth
        total_gt = row[1][1]

        img = cv2.imread(arquivo)
        
        #converte para tons de cinza
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        #aplica o filtro
        gray_blur = noise_reduction(gray, f)
        
        #obtem o número de linhas da imagem
        rows = gray_blur.shape[0]
        
        #aplica a transformada com parâmetros pré-definidos
        circles = cv2.HoughCircles(gray_blur,cv2.HOUGH_GRADIENT,1, rows/8,
                                param1=param1_esc,param2=param2_esc,
                                minRadius=minRadius_esc,maxRadius=maxRadius_esc)
        
        #estima a quantidade de círculos
        qtd_circles = len(circles[0])

        #adicionar dados na lista para o dataframe
        resultado.append([f, arquivo, total_gt, qtd_circles, (qtd_circles-total_gt)/total_gt])

#salvar o dataframe em formato csv
res_df = pd.DataFrame(resultado,columns=['filtro','nome_arquivo',
                                         'ground_truth','qtd_encontrada','margem_erro'])
res_df.to_csv('resultado_filtros.csv')

#criar um dataframe com base na lista
print('Lista criada')

In [ ]:
res_df.head()
# Calcula o mean squared error para cada filtro aplicado
for f in filter_set:
    Y_true = res_df[res_df['filtro']==f]['ground_truth']
    Y_pred = res_df[res_df['filtro']==f]['qtd_encontrada']
    MSE = np.square(np.subtract(Y_true,Y_pred)).mean()
    variance = np.var(np.subtract(Y_true,Y_pred))
    print('MSE para o filtro {filtro}: {mse}'.format(filtro=f,mse=MSE))
    print('Variância para o filtro {filtro}: {var}'.format(filtro=f,var=variance))

In [ ]:
#função para marcar os círculos detectados na imagem original
#retorna uma imagem com os círculos desenhados em verde, numerados
#em vermelho no centro de cada círculo
def show_detected_circles(img_source, circles, color_circle):
    
    img = np.copy(img_source)
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # fontScale
    fontScale = 1

    # Blue color in BGR
    color_text = (255, 0, 0)

    # Line thickness of 2 px
    thickness = 2
    if not (circles is None):
        circles = np.uint16(np.around(circles))
        v = 0
        for i in circles[0,:]:
            x = i[0]
            y = i[1]
            raio = i[2]
            # draw the outer circle
            cv2.circle(img,(x,y),raio,color_circle, 2)

            #print('Circulo: %d , raio: %0.2f Centro:(%d,%d)'% (v,raio,x,y))

            # org
            org = (x-12, y+12)

            #identifica o circulo com o numero do mesmo
            img = cv2.putText(img, str(v), org, font, 
                       fontScale, color_text, thickness, cv2.LINE_AA)

            v += 1
    return img

# Ler anotações para marcar os bounding boxes em uma imagem arbitrária

In [ ]:
local = pd.read_csv('localizacao_celulas.csv')

In [ ]:
#Seleciona as linhas que possuem apenas o caso de teste para RBC
#Objetivo: gerar as anotações no arquivo para exibição do ground truth
imagem0_locais = local[local['filename'] == img_selecionada]
imagem0_locais = imagem0_locais[imagem0_locais['cell_type']=='RBC']

In [ ]:
imagem0_locais.head()
imagem0_locais.count()

In [ ]:
img_annotated = img_rgb.copy()
for indice, linha in imagem0_locais.iterrows():
    #print(linha['xmin'], linha['xmax'], linha['ymin'],linha['ymax'])
    cv2.rectangle(img_annotated,(linha['xmin'],linha['ymin']),
                                (linha['xmax'],linha['ymax']),(0,255,0),2)

fig3, ax3 = plt.subplots(1,2,figsize=(12,16))
ax3[0].imshow(img_rgb)
ax3[0].set_title('Imagem Original')
ax3[1].imshow(img_annotated)
ax3[1].set_title('Imagem marcando as RBCs')
plt.show()

In [ ]:
# Executar a Transformada de Hough para o dataset, gerando um arquivo de resultado
# e imagens de output para visualização/inspeção

In [ ]:
# observar o quantitativo já preparado no arquivo quantidade_celulas_rbc.csv
df_rbc.head()

In [ ]:
#Gerar as imagens para os parâmetros escolhidos. Gravar também os 
#resultados obtidos em dataframe para comparação
resultado = []
for row in df_rbc.iterrows():

    #percorre cada arquivo do dataframe
    nome_arquivo = row[1][0]
    arquivo = caminho_dataset + row[1][0]

    #para este arquivo anota o ground truth
    total_gt = row[1][1]

    img = cv2.imread(arquivo)

    #converte para tons de cinza
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #aplica o filtro
    gray_blur = noise_reduction(gray, filtro_esc)

    #obtem o número de linhas da imagem
    rows = gray_blur.shape[0]

    #aplica a transformada com parâmetros pré-definidos
    circles = cv2.HoughCircles(gray_blur,cv2.HOUGH_GRADIENT,1, rows/8,
                            param1=param1_esc,param2=param2_esc,
                            minRadius=minRadius_esc,maxRadius=maxRadius_esc)

    #estima a quantidade de círculos
    qtd_circles = len(circles[0])
        
    #adicionar dados na lista para o dataframe
    resultado.append([filtro_esc, nome_arquivo, total_gt, qtd_circles, 
                      (qtd_circles-total_gt)/total_gt])

    #marca os círculos em verde, identificando-os
    img2 = show_detected_circles(img, circles, (0,255,0))
    
    #salva em arquivo
    arquivo_w = caminho_output +'/com_wbc/'+ row[1][0]
    cv2.imwrite(arquivo_w, img2)
    
#criar um dataframe com base na lista
print('Arquivos gerados na pasta:', caminho_output + 'com_wbc')

In [ ]:
res_df = pd.DataFrame(resultado,columns=['filtro','nome_arquivo','ground_truth',
                                         'qtd_encontrada','margem_erro'])

In [ ]:
res_df.head()

In [ ]:
res_df.to_csv('resultado_final_com_wbc.csv')

In [ ]:
gt = res_df['ground_truth'][:20]
qtd = res_df['qtd_encontrada'][:20]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = np.arange(len(gt))
width = 0.35
rects1 = ax.bar(x - width/2,gt, width, label='Referência')
rects2 = ax.bar(x + width/2,qtd,width,label='Qtd Encontrada')
ax.set_ylabel('Nº Células')
ax.legend(loc=4,framealpha=1)

ax.bar_label(rects1,padding=4)
ax.bar_label(rects2,padding=4)
plt.show()
#ax.bar(X + 0.25,res_df['qtd_encontrada'], color = 'g', width = 0.25)
#res_df.plot.bar(rot=0)

# Criação de imagens exemplo de detecção Canny
Foram escolhidos parâmetros 10, 20 e 60 como threshold máximo

In [ ]:
item = df_rbc['filename'][23]
arquivo = caminho_dataset + item
img = cv2.imread(arquivo)
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
blue,green,red = cv2.split(img)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray_blur = cv2.GaussianBlur(gray,(3,3),cv2.BORDER_DEFAULT)

c1 = cv2.Canny(gray_blur,1, 10)
c2 = cv2.Canny(gray_blur,1, 20)
c3 = cv2.Canny(gray_blur,1, 60)


fig3, ax4 = plt.subplots(1,3,figsize=(20,24))
ax4[0].imshow(c1)
ax4[0].set_title('Canny com maxValue 10')

ax4[1].imshow(c2)
ax4[1].set_title('Canny com maxValue 20')

ax4[2].imshow(c3)
ax4[2].set_title('Canny com maxValue 60')
plt.show()

# Etapa de remoção das células brancas

In [ ]:
from skimage import measure, __version__
from skimage.color import label2rgb

In [ ]:
def create_circles_mask(img_source, circles):
    
    img = np.zeros((img_source.shape[0],img_source.shape[1]))

    white_color = (2255,255,255)
    
    # Line thickness of 2 px
    thickness = 2
    if not (circles is None):
        circles = np.uint16(np.around(circles))
        v = 0
        for i in circles[0,:]:
            x = i[0]
            y = i[1]
            raio = i[2]
            # draw the outer circle
            cv2.circle(img,(x,y),raio,white_color,-1)

            v += 1
    return img

In [ ]:
arquivo = '../dataset/BCCD/JPEGImages/' + img_selecionada

#abre o arquivo
img = cv2.imread(arquivo)

#converte para tons de cinza
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#aplica o filtro
gray_blur = noise_reduction(gray, filtro_esc)

#obtem o número de linhas da imagem
rows = gray_blur.shape[0]

#aplica a transformada com parâmetros pré-definidos
circles = cv2.HoughCircles(gray_blur,cv2.HOUGH_GRADIENT,1, rows/8,
                        param1=param1_esc,param2=param2_esc,
                        minRadius=minRadius_esc,maxRadius=maxRadius_esc)

#cria uma máscara da imagem
img_mask = create_circles_mask(gray, circles)

#coloca os labels
img_label = measure.label(img_mask,connectivity=img_mask.ndim)
#plt.imshow(img_label)

#aplica um overlay
img_label_ovl = label2rgb(img_label, image=img)
plt.imshow(img_label_ovl)

#properties = ['label','mean_intensity','solidity','perimeter','area']
#props = measure.regionprops_table(img_label, img, properties)

In [ ]:
def get_bbox(circles):
    b = []
    circles = np.uint16(np.around(circles))

    for c in circles[0,:]:
        xc = c[0]
        yc = c[1]
        r  = c[2]

        if xc >= r:
            x1 = xc - r
        else:
            x1 = 0
        
        if yc >= r:
            y1 = yc - r
        else:
            y1 = 0
        
        x2 = xc + r
        y2 = yc + r

        if x2 < 0:
            x2 = 0

        if y2 < 0:
            y2 = 0
            
        b.append([(x1,y1),(x2,y2)])
        
    return b

In [ ]:
print(circles)
a = get_bbox(circles)
a[0][1:3]


In [ ]:
def save_bbox_img(img_source, b):
    img = np.copy(img_source)
    file = '../output/celula'
    v = 0
    for qua in b:
        v += 1
        x1, y1 = qua[0]
        x2, y2 = qua[1]

        #extrai o segmento de imagem
        img_seg = get_bbox_img(img, qua)

        cv2.imwrite(file+str(v)+'.jpg',img_seg)
    return
        

In [ ]:
def get_bbox_img(img_source, bbox):
    img = np.copy(img_source)
    x1, y1 = bbox[0]
    x2, y2 = bbox[1]
    return img[y1:y2,x1:x2,:]

In [ ]:
save_bbox_img(img, a)

In [ ]:
fig5, ax5 = plt.subplots(3,3, constrained_layout=True, sharey=True,figsize=(12,8))
file = '../output/celula'
im=[]
v = 0
li = 0
co = 0
for i in range(12):
    v += 1
    if v > 9:
        break
    arquivo = file+str(v)+'.jpg'
    #print(arquivo)
    #im.append(cv2.imread(arquivo))
    im = cv2.imread(arquivo)
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    ax5[li][co].imshow(im)
    co += 1
    if co == 3:
        li += 1
        co = 0
plt.show()

In [ ]:
ax = []
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
for q in a:
    i = get_bbox_img(hsv, q)
    h, s, v = cv2.split(i)
    ax.append(v.ravel().mean())
df_ax = pd.Series(ax)
df_ax.describe()

In [ ]:
ax = []
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cont = 0
for q in a:
    cont += 1
    i = get_bbox_img(hsv, q)

    h, s, v = cv2.split(i)
    
    lower_mask = h[:,:] > 115
    upper_mask = h[:,:] < 130
    
    mask = lower_mask*upper_mask
    qtd_pixels = np.sum(mask)
    print('Recorte {cont}&{pixels}'.format(cont=cont,pixels=qtd_pixels))
    ax.append(qtd_pixels)
    
df_blue_pixels = pd.DataFrame(ax)

In [ ]:
df_ax = pd.Series(ax)
df_ax.head()

In [ ]:
#Reprodução do código de máscara para a célula branca
arquivo = '../output/celula1.jpg'
img = cv2.imread(arquivo)

#converte para o espaço HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#realiza o split dos canais
h, s, v = cv2.split(hsv)

#define os lower/upper bounds
lower_mask = h[:,:] > 115 #valores predefinidos
upper_mask = h[:,:] < 130 #para upper e lower

mask = lower_mask * upper_mask #multiplicação realiza um AND

fig6, ax6 = plt.subplots(1,3, constrained_layout=True, sharey=True,figsize=(12,8))

ax6[0].imshow(img)  ##imagem original
ax6[0].set_title('Imagem original')

ax6[1].imshow(mask,cmap='gray')  ##máscara aplicada
ax6[1].set_title('Máscara em formato binário')

red = img[:,:,0]*mask #filtra dentro dos canais
green = img[:,:,1]*mask
blue = img[:,:,2]*mask


img2 = cv2.merge([red,green,blue])
ax6[2].imshow(img2)
ax6[2].set_title('Filtro de máscara aplicado sobre a imagem')
plt.show()

In [ ]:
print(mask)

In [ ]:
#Gerar as imagens para os parâmetros escolhidos. Gravar também os 
#resultados obtidos em dataframe para comparação
resultado = []
for row in df_rbc.iterrows():

    #percorre cada arquivo do dataframe
    nome_arquivo = row[1][0]
    arquivo = caminho_dataset + row[1][0]

    #para este arquivo anota o ground truth
    total_gt = row[1][1]

    img = cv2.imread(arquivo)

    #converte para tons de cinza
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #aplica o filtro
    gray_blur = noise_reduction(gray, filtro_esc)

    #obtem o número de linhas da imagem
    rows = gray_blur.shape[0]

    #aplica a transformada com parâmetros pré-definidos
    circles = cv2.HoughCircles(gray_blur,cv2.HOUGH_GRADIENT,1, rows/8,
                            param1=param1_esc,param2=param2_esc,
                            minRadius=minRadius_esc,maxRadius=maxRadius_esc)

    #estima a quantidade de círculos total
    qtd_circles = len(circles[0])

    #quantifica as células brancas (wbc)
    bbox = get_bbox(circles)  #obtem os bounding boxes das celulas

    #converte a imagem original para HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    qtd_wbc = qtd_rbc = 0
    circles_wbc = []
    circles_rbc = []
    cont = 0
    for bb in bbox:
        ibb = get_bbox_img(hsv, bb) #faz um crop da imagem no espaço de cor HSV

        h, s, v = cv2.split(ibb) # faz o split, separando os canais

        lower_mask = h[:,:] > 115 #aplica lower e upper pré-definidos
        upper_mask = h[:,:] < 130

        mask = lower_mask*upper_mask  #define a mascara total
        qtd_pixels = np.sum(mask)     #soma a quantidade de pixels
        
        xc = circles[0][cont][0]
        yc = circles[0][cont][1]
        raio = circles[0][cont][2]
        
        if qtd_pixels > 1000:  #quantidade predefinida de pixels azuis para wbc
            qtd_wbc += 1 #incrementa wbc
            circles_wbc.append(True)
            circles_rbc.append(False)
        else:
            qtd_rbc += 1 #incrementa rbc
            circles_wbc.append(False)
            circles_rbc.append(True)
        #incrementa contador de circulos
        cont += 1

    c_wbc = np.array([circles[0][circles_wbc]])
    c_rbc = np.array([circles[0][circles_rbc]])


    #adicionar dados na lista para o dataframe
    resultado.append([filtro_esc, nome_arquivo, total_gt, qtd_circles, qtd_wbc, qtd_rbc, \
                      (qtd_rbc-total_gt)/total_gt])

    #marca os círculos das celulas vermelhas em verde, identificando-os
    color = (0,255,0)
    img2 = show_detected_circles(img, c_rbc, color)
    
    if qtd_wbc > 0:
        #marca os círculos das células brancas em amarelo, indentificando-os
        color = (255,255,0)
        img2 = show_detected_circles(img2, c_wbc, color)    
    
    #salva em arquivo
    arquivo_w = caminho_output +'/sem_wbc/'+ row[1][0]
    cv2.imwrite(arquivo_w, img2)
    
#criar um dataframe com base na lista
print('Arquivos gerados na pasta:', caminho_output + 'sem_wbc')

In [ ]:
res_df2 = pd.DataFrame(resultado,columns=['filtro','nome_arquivo',
                                          'ground_truth','qtd_encontrada',
                                          'qtd_wbc','qtd_rbc','margem_erro'])

In [ ]:
res_df2.head()

In [ ]:
#define a acuracia para cada imagem
res_df2['acuracia'] = res_df2['qtd_encontrada']/res_df2['ground_truth']

In [ ]:
#exibe as estatisticas para os resultados encontrados de acuracia (media, dp, mediana, etc)
res_df2['acuracia'].describe()

In [ ]:
#ao inves de trabalhar com acuracia, partimos para o uso da margem de erro
res_df2['margem_erro'].describe()

In [ ]:
desvio = 0.575844
print(desvio*1.5)
print(np.sum(res_df2['margem_erro']>(1.5*desvio)))
res_df2[res_df2['margem_erro']>(1.5*desvio)]

zero = np.sum(res_df2['margem_erro']==0)
maior_zero = np.sum(res_df2['margem_erro']>0)
menor_zero = np.sum(res_df2['margem_erro']<0)
print('Erro == 0..:',zero)
print('Erro > 0..:',maior_zero)
print('Erro < 0..:',menor_zero)
print('Total.....:',24+258+67+15)

In [ ]:
res_df2[res_df2['margem_erro']==0]

In [ ]:
plt.boxplot(res_df2['margem_erro'])

In [ ]:
b = np.uint8(res_df2.shape[0]/4)
plt.hist(res_df2['margem_erro'],bins=b)
plt.show()

In [ ]:
import scipy
mean = 0.352883
standard_deviation = 0.575844

x_values = np.arange(-5, 5, 0.1)
y_values = scipy.stats.norm(mean, standard_deviation)

plt.plot(x_values, y_values.pdf(x_values))

In [ ]:
np.sum(res_df2['acuracia']<1)

In [ ]:
plt.boxplot(res_df2['acuracia'])

In [ ]:
res_df2[res_df2['acuracia']>2.29][['nome_arquivo','acuracia']]

In [ ]:
res_df2[res_df2['nome_arquivo']=='BloodImage_00270.jpg']

In [ ]:
res_df2.to_csv('resultado_final_sem_wbc.csv')

In [ ]:
gt = res_df2['ground_truth'][:20]
qtd = res_df2['qtd_rbc'][:20]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = np.arange(len(gt))
width = 0.35
rects1 = ax.bar(x - width/2,gt, width, label='Referência')
rects2 = ax.bar(x + width/2,qtd,width,label='Qtd Encontrada')
ax.set_ylabel('Nº Células')
ax.legend(loc=4,framealpha=1)

ax.bar_label(rects1,padding=4)
ax.bar_label(rects2,padding=4)
plt.show()

In [ ]:
#montar imagem comparativa para o arquivo BloodImage_000270.jpg

#arquivos_experimento = ['BloodImage_00023.jpg']
caminho_dataset = '../dataset/BCCD/JPEGImages/'

#escolha de uma imagem específica para visualização e experimentação
img_selecionada = 'BloodImage_00270.jpg'
arquivo = '../dataset/BCCD/JPEGImages/' + img_selecionada

img270 = cv2.imread('../output/sem_wbc/' + img_selecionada)
img270 = cv2.cvtColor(img270, cv2.COLOR_BGR2RGB)

#carrega a imagem original para fazer a anotação
img_orig = cv2.imread(caminho_dataset + img_selecionada)
img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

#obter as anotações 
imagem0_locais = local[local['filename'] == img_selecionada]
#imagem0_locais = imagem0_locais[imagem0_locais['cell_type']=='RBC']

img_annotated = img_orig.copy()
for indice, linha in imagem0_locais.iterrows():
    #print(linha['xmin'], linha['xmax'], linha['ymin'],linha['ymax'])
    if linha['cell_type'] == 'RBC':
        color = (0,255,0)
    elif linha['cell_type'] == 'WBC':
        color = (0,255,255)
    elif linha['cell_type'] == 'Platelets':
        color = (0,0,255)
    cv2.rectangle(img_annotated,(linha['xmin'],linha['ymin']),
                                (linha['xmax'],linha['ymax']),color,2)

fig3, ax3 = plt.subplots(1,2,figsize=(16,22))
ax3[0].imshow(img270)
ax3[0].set_title('(a) círculos detectados pela Transformada de Hough' )
ax3[1].imshow(img_annotated)
ax3[1].set_title('(b) anotações do dataset original')
plt.show()
